### Final Project Submission

Please fill out:
* Student name: Brian Lee
* Student pace: Full time
* Scheduled project review date/time: --
* Instructor name: Victor
* Blog post URL: -- 

# Movie Analysis

## Business Problem

Summary of the business problem you are trying to solve, and the data questions that you plan to answer to solve them.

***

Questions to consider:

* What are the business's pain points related to this project?
* How did you pick the data analysis question(s) that you did?
* Why are these questions important from a business perspective?

In [ ]:
## Write thoughts here:

# Figure out what movies do well at the box office for Microsoft
#  * Movie Genre
#  * Does actors/directors play into "success"?
#  * Movie cost to return

## Using info from IMDB, Rotten Tomatoes, etc, figure out with data what will play into "success"
#  Use Data visualizations to help prove point

## Data Understanding

Describe the data being used for this project.

Movie datasets from: 

* Box Office Mojo
* IMDB
* Rotten Tomatoes 
* TheMovieDB
* The Numbers

Minimum datasets to use:

* imdb.title.basics
* imdb.title.ratings
* bom.movie_gross
***

Questions to consider:

* Where did the data come from, and how do they relate to the data analysis questions?
* What do the data represent? Who is in the sample and what variables are included?
* What is the target variable?
* What are the properties of the variables you intend to use?

In [3]:
# Import standard packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

%matplotlib inline

### IMDB data

In [4]:
# May be good to query the databases with sqlite3 as they contain various keys to connect various tables (after clearning data)
title_basics = pd.read_csv('zippedData/imdb.title.basics.csv.gz')
title_basics.head()

,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [5]:
title_ratings = pd.read_csv('zippedData/imdb.title.ratings.csv.gz')
title_ratings.head()

,tconst,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


### Box Office Mojo

In [6]:
movie_gross = pd.read_csv('zippedData/bom.movie_gross.csv.gz')
movie_gross.head()
# display(len(movie_gross))
# movie_gross.isna().sum()

# around 1/3 of foreign_gross is missing.
# should do a separate analysis for domestic/foreign when graphing. domestic/foreign success is not 100% correlated

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


### Rotten Tomatoes data

In [7]:
movie_info = pd.read_csv('zippedData/rt.movie_info.tsv.gz', delimiter="\t")
movie_info.head()
movie_info.isna().sum()

id                 0
synopsis          62
rating             3
genre              8
director         199
writer           449
theater_date     359
dvd_date         359
currency        1220
box_office      1220
runtime           30
studio          1066
dtype: int64

In [8]:
# try:
#     movie_reviews = pd.read_csv('zippedData/rt.reviews.tsv.gz', delimiter="\t")
#     movie_reviews.head()
# except Exception as e:
#     print(e)

In [9]:
rt_movie_info_df=pd.read_csv('zippedData/rt.movie_info.tsv.gz', delimiter='\t', header=0)
rt_movie_info_df.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


In [10]:
rt_reviews_df=pd.read_csv('zippedData/rt.reviews.tsv.gz',delimiter='\t',header=0, encoding='latin-1')
rt_reviews_df.head()

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


In [11]:
# Unusable data - no related titles. Will opt to not use data sets from Rotten Tomatoes

### The Numbers data

In [12]:
## Movie budgets - The Numbers
# Has production budget, domestic/worldwide gross

movie_budgets = pd.read_csv('zippedData/tn.movie_budgets.csv.gz')
movie_budgets.head()
# movie_budgets.isna().sum() # no missing values

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [13]:
# Titles will be dificult to compare with that of others. Maybe better to create a separate visualization with purely this data.
# Will need to merge the genres to this table in order to get the correct information

## Data Preparation

Describe and justify the process for preparing the data for analysis.

***

Questions to consider:

* Were there variables you dropped or created?
* How did you address missing values or outliers?
* Why are these choices appropriate given the data and the business problem?

### IMDB Data

* We need to merge data from tables using the primary key of 'tconst'
* Clean data for duplicates and NaN items
* We need to separate and create new columns, or create list with genres options for each movie

In [79]:
# Merging datasets using primary key of 'tconst'
imdb_df = title_basics.merge(title_ratings, how="inner", on='tconst')

In [85]:
# Need to split genres into a searchable list and reinsert OR can make list of genres be columns with 0,1 indicator
length = range(len(imdb_df['genres']))
for i in length:
    if type(imdb_df.loc[i,'genres']) == str:
        imdb_df.at[i, 'genres'] = imdb_df.loc[i, 'genres'].split(',')
    else:
        continue
imdb_df

,tconst,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"[Action, Crime, Drama]",7.0,77
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"[Biography, Drama]",7.2,43
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,[Drama],6.9,4517
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"[Comedy, Drama]",6.1,13
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"[Comedy, Drama, Fantasy]",6.5,119
...,...,...,...,...,...,...,...,...
73851,tt9913084,Diabolik sono io,Diabolik sono io,2019,75.0,[Documentary],6.2,6
73852,tt9914286,Sokagin Çocuklari,Sokagin Çocuklari,2019,98.0,"[Drama, Family]",8.7,136
73853,tt9914642,Albatross,Albatross,2017,NaN,[Documentary],8.5,8
73854,tt9914942,La vida sense la Sara Amat,La vida sense la Sara Amat,2019,NaN,NaN,6.6,5


In [86]:
# missing data in runtime_minutes and genres. We probably want to remove items without genre as it is defining trait
# imdb_df['genres'].unique()

imdb_df.dropna(subset=['genres'], inplace=True)
imdb_df.isna().sum()

tconst                0
primary_title         0
original_title        0
start_year            0
runtime_minutes    7332
genres                0
averagerating         0
numvotes              0
dtype: int64

In [87]:
# We do not need the 'tconst' and 'runtime_minutes' in our data set. Will drop columns
imdb_df.drop(['tconst','runtime_minutes'],axis=1, inplace=True)
imdb_df.head()

,primary_title,original_title,start_year,genres,averagerating,numvotes
0,Sunghursh,Sunghursh,2013,"[Action, Crime, Drama]",7.0,77
1,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,"[Biography, Drama]",7.2,43
2,The Other Side of the Wind,The Other Side of the Wind,2018,[Drama],6.9,4517
3,Sabse Bada Sukh,Sabse Bada Sukh,2018,"[Comedy, Drama]",6.1,13
4,The Wandering Soap Opera,La Telenovela Errante,2017,"[Comedy, Drama, Fantasy]",6.5,119


In [89]:
imdb_df.set_index('primary_title',inplace=True)

KeyError: "None of ['primary_title'] are in the columns"

In [16]:
# Check again for missing data items
imdb_df.isna().sum()

primary_title     0
original_title    0
start_year        0
genres            0
averagerating     0
numvotes          0
dtype: int64

In [90]:
# Final check on dataset
imdb_df

,original_title,start_year,genres,averagerating,numvotes
primary_title,,,,,
Sunghursh,Sunghursh,2013,"[Action, Crime, Drama]",7.0,77
One Day Before the Rainy Season,Ashad Ka Ek Din,2019,"[Biography, Drama]",7.2,43
The Other Side of the Wind,The Other Side of the Wind,2018,[Drama],6.9,4517
Sabse Bada Sukh,Sabse Bada Sukh,2018,"[Comedy, Drama]",6.1,13
The Wandering Soap Opera,La Telenovela Errante,2017,"[Comedy, Drama, Fantasy]",6.5,119
...,...,...,...,...,...
Swarm Season,Swarm Season,2019,[Documentary],6.2,5
Diabolik sono io,Diabolik sono io,2019,[Documentary],6.2,6
Sokagin Çocuklari,Sokagin Çocuklari,2019,"[Drama, Family]",8.7,136


## Data Modeling

Describe and justify the process for analyzing or modeling the data.

***

Questions to consider:
* How did you analyze or model the data?
* How did you iterate on your initial approach to make it better?
* Why are these choices appropriate given the data and the business problem?

In [ ]:
# Here you run your code to model the data

## Evaluation

Evaluate how well your work solves the stated business problem.

***

Questions to consider:

* How do you interpret the results?
* How well does your model fit your data? How much better is this than your baseline model?
* How confident are you that your results would generalize beyond the data you have?
* How confident are you that this model would benefit the business if put into use?

In [ ]:
# Write thoughts here:

## Conclusions

Provide your conclusions about the work you've done, including any limitations or next steps.

***

Questions to consider:

* What would you recommend the business do as a result of this work?
* What are some reasons why your analysis might not fully solve the business problem?
* What else could you do in the future to improve this project?

In [ ]:
# Write thoughts here:

## Next Steps: